# Internet of Things (2IMN25)
### Machine Learning Assignment

### Introduction

The objective of the assignment is to use supervised machine learning techniques to make predictions about the net energy consumed based on the weather conditions. This will be accomplished by training a regression model on (real) data obtained from the weather sensor and energy meter over this year. This model will then be used to make predictions about the net energy consumption for data-points not *seen* by the model during training.

In [1]:
""" Import all libraries here """
import numpy as np
import os
import math
from datetime import datetime
import pandas as pd
from dateutil import parser
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

## Reading the dataset

In the code box below, please read in the data-sets into suitable data structures. The key task here would be to contruct the feature vectors which would be fed to the regression algorithm.

The datasets for the weather, rain and energy consumption have been provided in the zip file, extract this zip file into the same folder as the python notebook. The function should read both the weather and rainfall datafiles and create a merged feature vector array based on time-stamps.



In [2]:
def merge_weather_rain_data(data_path): # Select month for which data is collected
    """ 
    Reads in the CSV files for weather and rain data
    Merged them into a single array based on matching time-stamps
    
    """
        
    """ Read in the weather"""
    weatherFrame = pd.read_csv(os.path.join(data_path,'Weather-Eindhoven.csv'),
                             delimiter = ',',
                             header=0,
                             parse_dates = [0],
                             usecols = [0,1,2,4,6,7,8]) #Drop "chill", too many missing values
    weather_array  = weatherFrame.as_matrix()

    """ Read in the rain data"""
    rainFrame = pd.read_csv(os.path.join(data_path,'Rain-Best.csv'),
                             delimiter = ',',
                             header =  None,
                             skiprows = 1,
                             index_col = None,
                             parse_dates = [0]
                            )
    rain_array = rainFrame.as_matrix()

    """ Synchronize the 2 data-sets by removing the extra data from the rainfall dataset"""
    remove_rows = []
    for element,idx in zip(rain_array[:,0],range(len(rain_array[:,0]))):
        if element.minute%10 != 0:
            remove_rows.append(idx)

    synced_rain_array = np.delete(rain_array,remove_rows,axis = 0)

    print(synced_rain_array.shape)
    print(weather_array[:,0].shape)

    """ Merge the weather and rain data when time-stamps are equal """
    merged_array = []
    for rainElem in synced_rain_array:
        try :
            merged_row = []
            valid_row = list(weather_array[:,0]).index(rainElem[0])
            for weatherFeature in weather_array[valid_row]:
                merged_row.append(weatherFeature)
            merged_row.append(rainElem[1])
            merged_row.append(weather_array[valid_row][0].hour) # Add "time of day" information 
            merged_array.append(np.asarray(merged_row))
        except ValueError:
            print('Did not find matching timestamp in the weather data, continuing')
            continue

    merged_array = np.asarray(merged_array)
    return merged_array

# Create the merged array
data_path = os.path.join(os.getcwd(),'data')
merged_array = merge_weather_rain_data(data_path)

(43844, 2)
(43872,)
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matchin

## Cleaning the data

The datafiles may contain missing entries, in such an event, it is needed to get rid of such datapoints. The function should take in the created feature vector array and delete rows if features are missing.


In [3]:

def clean_up(merged_array):
    """ 
    Clean up the data by removing rows with missing data points 
    
    """
    missing_data_rows = []
    for row,idx in zip(merged_array,range(merged_array.shape[0])):
        for elem in row[1:]: #Timestamp,Description cannot be "checked"
            if pd.isnull(elem):
                missing_data_rows.append(idx)
                break # To prevent the same row from being added multiple times to the list, break when the first "nan" is found
    clean_data_array = np.delete(merged_array,missing_data_rows,axis=0)
    return clean_data_array

def filter_by_month(data_array,month):
    """
    Filter weather data by month
    
    """
    rows_delete = []
    for row,idx in zip(data_array,range(data_array.shape[0])):
        if row[0].month != month and row[0].month != (month+1) and row[0].month != (month+2):
            rows_delete.append(idx)
    filtered_array = np.delete(data_array,rows_delete,axis=0)
    return filtered_array

clean_data_array = clean_up(merged_array)
month_array = filter_by_month(clean_data_array,month = 7)
for data in month_array:
    if (data[0].month == 8 and data[0].day == 8):
        print(data)

[Timestamp('2017-08-08 00:00:00') 'zwaar bewolkt' 15.4 82.0 3.0 22700.0
 1013.9 0.0 0]
[Timestamp('2017-08-08 00:10:00') 'geheel bewolkt' 15.6 81.0 3.0 23800.0
 1013.8 0.0 0]
[Timestamp('2017-08-08 00:20:00') 'geheel bewolkt' 15.7 81.0 2.0 26800.0
 1013.7 0.0 0]
[Timestamp('2017-08-08 00:30:00') 'zwaar bewolkt' 15.8 81.0 3.0 24900.0
 1013.4 0.0 0]
[Timestamp('2017-08-08 00:40:00') 'half bewolkt' 15.8 80.0 3.0 23800.0
 1013.1 0.0 0]
[Timestamp('2017-08-08 00:50:00') 'zwaar bewolkt' 15.9 80.0 3.0 23100.0
 1012.9 0.0 0]
[Timestamp('2017-08-08 01:00:00') 'half bewolkt' 15.8 80.0 3.0 21800.0
 1012.8 0.0 1]
[Timestamp('2017-08-08 01:10:00') 'half bewolkt' 15.7 80.0 3.0 20100.0
 1012.6 0.0 1]
[Timestamp('2017-08-08 01:20:00') 'licht bewolkt' 15.6 81.0 3.0 18800.0
 1012.4 0.0 1]
[Timestamp('2017-08-08 01:30:00') 'licht bewolkt' 15.6 81.0 3.0 19800.0
 1012.2 0.0 1]
[Timestamp('2017-08-08 01:40:00') 'half bewolkt' 15.6 81.0 3.0 21000.0
 1012.2 0.0 1]
[Timestamp('2017-08-08 01:50:00') 'half bewol

## Reading in the energy values

The aim of the assignment is to predict the solar energy produced based on the weather conditions. Thus, for each weather datapoint, the solar energy must be calclulated using the energy datafiles and matching the time-stamps. Define a function that first creates a single datastructure from all the different energy files. Then, design a function that can perform an energy look-up based on the time-stamp of the weather data. This fucntion should return the total solar energy produced between the previous timestamp and current timestamp.Using this look-up function, loop over all the time-stamps of the weather data and generate energy labels. It should be noted, that for certain time-stamps, the energy values might be missing, datapoints corresponding to such timestamps must be deleted from the data array. 

*Hint : Check the documentation for the python package pandas to design a fast lookup function*

In [4]:
def create_dataframe(energyDir):
    """ 
    Returns a dataframe that is formed by concatenating all the energy data files 
    """
    fileList = []
    energyDir = os.path.join(os.getcwd(),energyDir)
    dirList = [os.path.join(energyDir, o) for o in os.listdir(energyDir) 
                    if os.path.isdir(os.path.join(energyDir,o))]
    
    sortedDirList = sorted(dirList)
    for direc in sortedDirList:
        dirFiles = [os.path.join(direc, f) for f in os.listdir(direc) 
                    if os.path.isfile(os.path.join(direc,f))]
        for f in dirFiles:
            fileList.append(f)
        
    fileList = sorted(fileList)
    frame = pd.DataFrame()
    frameList = []
    for f in fileList:
        try:
            df = pd.read_csv(f,index_col=None, header=None,parse_dates=[[0,1]],skipfooter=2)
            frameList.append(df)
        except:
            continue
    frame = pd.concat(frameList)
    sparseFrame = frame.drop([2,3,4,5,6,7,8,9,10,11,13],axis=1)
    sparseFrame.columns = ['TimeStamp','Solar Energy Produced']
    return sparseFrame
    
def energy_frame_month(energyDir,month):
    """
    Create an energy frame for only the specified month
    
    """
    energyDir = os.path.join(os.getcwd(),energyDir)
    if month < 10:
        monthDir = os.path.join(energyDir,'0{}'.format(month))
    else:
        monthDir = os.path.join(energyDir,'{}'.format(month))
        
    dirFiles = [os.path.join(monthDir, f) for f in os.listdir(monthDir) 
                    if os.path.isfile(os.path.join(monthDir,f))]
    fileList = []
    for f in dirFiles:
        fileList.append(f)
        fileList = sorted(fileList)
    frame = pd.DataFrame()
    frameList = []
    for f in fileList:
        try:
            df = pd.read_csv(f,index_col=None, header=None,parse_dates=[[0,1]],skipfooter=2)
            frameList.append(df)
        except:
            continue
    frame = pd.concat(frameList)
    sparseFrame = frame.drop([2,3,4,5,6,7,8,9,10,11,13],axis=1)
    sparseFrame.columns = ['TimeStamp','Solar Energy Produced']
    return sparseFrame

    
def multi_month_frame(start_month,num_months):
    """
    Concatenates for a given number of months w.r.t a start month
    
    """
    frameList = []
    for step in range(num_months):
        frameList.append(energy_frame_month('data/2017',month = start_month+step))
    return pd.concat(frameList)

energyFrame_conc = multi_month_frame(start_month = 7,num_months = 3)
energyFrame_conc[['Solar Energy Produced']] = energyFrame_conc[['Solar Energy Produced']].apply(pd.to_numeric,errors='ignore')
print(energyFrame_conc)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


               TimeStamp Solar Energy Produced
0    2017-07-01 00:00:01                     0
1    2017-07-01 00:00:11                     0
2    2017-07-01 00:00:21                     0
3    2017-07-01 00:00:31                     0
4    2017-07-01 00:00:41                     0
5    2017-07-01 00:00:51                     0
6    2017-07-01 00:01:01                     0
7    2017-07-01 00:01:11                     0
8    2017-07-01 00:01:21                     0
9    2017-07-01 00:01:31                     0
10   2017-07-01 00:01:41                     0
11   2017-07-01 00:01:51                     0
12   2017-07-01 00:02:01                     0
13   2017-07-01 00:02:11                     0
14   2017-07-01 00:02:21                     0
15   2017-07-01 00:02:31                     0
16   2017-07-01 00:02:41                     0
17   2017-07-01 00:02:51                     0
18   2017-07-01 00:03:01                     0
19   2017-07-01 00:03:11                     0
20   2017-07-

In [5]:
def energy_lookup(data_array,energyFrame,idx):
    """ 
    Given the index, calculated the net-energy consumed in the 10-minute interval preceeding current time
    
    """
    subFrame = energyFrame[(energyFrame['TimeStamp']>=data_array[idx-1][0]) & (energyFrame['TimeStamp']<data_array[idx][0])]
    if subFrame.shape[0] == 0:
        print(data_array[idx-1])
        print(data_array[idx])
        print(subFrame)
        return None
    solarList = []
    for solar in subFrame['Solar Energy Produced']:
        if isinstance(solar,str):
            continue
        solarList.append(solar)
    if len(solarList) == 0: #All strings?
        return None
    return np.sum(np.asarray(solarList).astype(float))


def create_labels(data_array,energyFrame):
    """
    Uses the look-up function to create labels (if possible)
    for every feature vector in the feature array.
    
    """
    
    missing_labels = []
    labels = []
    labels_solar = []
    labels.append(0)
    labels_solar.append(0)
    energyFrame['TimeStamp'] = pd.to_datetime(energyFrame['TimeStamp']) 
    for row,idx in zip(data_array,range(data_array.shape[0])):
        if idx == 0:
            continue
        netSolar = energy_lookup(data_array,energyFrame,idx)
        if netSolar is None:
            print('Missing label for row {}'.format(idx))
            missing_labels.append(idx)
        else:
            print('{} {}'.format(row[0],netSolar))
            labels_solar.append(netSolar)
    return labels_solar,missing_labels

labels_solar,missing_labels = create_labels(month_array,energyFrame_conc)
featureArray = np.delete(month_array,missing_labels,axis=0)
            

2017-07-01 00:10:00 0.0
2017-07-01 00:20:00 0.0
2017-07-01 00:30:00 0.0
2017-07-01 00:40:00 0.0
2017-07-01 00:50:00 0.0
2017-07-01 01:00:00 0.0
2017-07-01 01:10:00 0.0
2017-07-01 01:20:00 0.0
2017-07-01 01:30:00 0.0
2017-07-01 01:40:00 0.0
2017-07-01 01:50:00 0.0
2017-07-01 02:00:00 0.0
2017-07-01 02:10:00 0.0
2017-07-01 02:20:00 0.0
2017-07-01 02:30:00 0.0
2017-07-01 02:40:00 0.0
2017-07-01 02:50:00 0.0
2017-07-01 03:00:00 0.0
2017-07-01 03:10:00 0.0
2017-07-01 03:20:00 0.0
2017-07-01 03:30:00 0.0
2017-07-01 03:40:00 0.0
2017-07-01 03:50:00 0.0
2017-07-01 04:00:00 0.0
2017-07-01 04:10:00 0.0
2017-07-01 04:20:00 0.0
2017-07-01 04:30:00 0.0
2017-07-01 04:40:00 0.0
2017-07-01 04:50:00 0.0
2017-07-01 05:00:00 0.0
2017-07-01 05:10:00 0.0
2017-07-01 05:20:00 0.0
2017-07-01 05:30:00 0.0
2017-07-01 05:40:00 0.0
2017-07-01 05:50:00 0.0
2017-07-01 06:00:00 0.0
2017-07-01 06:10:00 0.0
2017-07-01 06:20:00 0.0
2017-07-01 06:30:00 1539.0
2017-07-01 06:40:00 3669.0
2017-07-01 06:50:00 3857.0
2017-07

2017-07-03 05:00:00 0.0
2017-07-03 05:10:00 0.0
2017-07-03 05:20:00 0.0
2017-07-03 05:30:00 0.0
2017-07-03 05:40:00 1134.0
2017-07-03 05:50:00 3616.0
2017-07-03 06:00:00 4114.0
2017-07-03 06:10:00 4455.0
2017-07-03 06:20:00 4528.0
2017-07-03 06:30:00 4317.0
2017-07-03 06:40:00 4047.0
2017-07-03 06:50:00 1124.0
2017-07-03 07:00:00 5424.0
2017-07-03 07:10:00 21968.0
2017-07-03 07:20:00 31695.0
2017-07-03 07:30:00 35608.0
2017-07-03 07:40:00 43223.0
2017-07-03 07:50:00 49149.0
2017-07-03 08:00:00 54804.0
2017-07-03 08:10:00 59944.0
2017-07-03 08:20:00 65071.0
2017-07-03 08:30:00 71037.0
2017-07-03 08:40:00 75461.0
2017-07-03 08:50:00 80975.0
2017-07-03 09:00:00 87092.0
2017-07-03 09:10:00 91426.0
2017-07-03 09:20:00 96510.0
2017-07-03 09:30:00 101669.0
2017-07-03 09:40:00 100890.0
2017-07-03 09:50:00 108348.0
2017-07-03 10:00:00 111405.0
2017-07-03 10:10:00 114352.0
2017-07-03 10:20:00 117821.0
2017-07-03 10:30:00 123670.0
2017-07-03 10:40:00 127126.0
2017-07-03 10:50:00 127234.0
2017-07-

2017-07-05 09:20:00 79816.0
2017-07-05 09:30:00 85463.0
2017-07-05 09:40:00 86442.0
2017-07-05 09:50:00 95392.0
2017-07-05 10:00:00 100145.0
2017-07-05 10:10:00 103169.0
2017-07-05 10:20:00 104669.0
2017-07-05 10:30:00 60356.0
2017-07-05 10:40:00 102644.0
2017-07-05 10:50:00 111386.0
2017-07-05 11:00:00 123383.0
2017-07-05 11:10:00 117108.0
2017-07-05 11:20:00 130450.0
2017-07-05 11:30:00 119843.0
2017-07-05 11:40:00 51520.0
2017-07-05 11:50:00 89268.0
2017-07-05 12:00:00 73782.0
2017-07-05 12:10:00 83822.0
2017-07-05 12:20:00 132757.0
2017-07-05 12:30:00 133135.0
2017-07-05 12:40:00 116445.0
2017-07-05 12:50:00 128752.0
2017-07-05 13:00:00 131849.0
2017-07-05 13:10:00 97797.0
2017-07-05 13:20:00 68788.0
2017-07-05 13:30:00 98533.0
2017-07-05 13:40:00 125193.0
2017-07-05 13:50:00 44055.0
2017-07-05 14:00:00 85545.0
2017-07-05 14:10:00 90552.0
2017-07-05 14:20:00 94966.0
2017-07-05 14:30:00 81024.0
2017-07-05 14:40:00 49435.0
2017-07-05 14:50:00 114175.0
2017-07-05 15:00:00 88816.0
2017

2017-07-07 13:30:00 114807.0
2017-07-07 13:40:00 143152.0
2017-07-07 13:50:00 75240.0
2017-07-07 14:00:00 67780.0
2017-07-07 14:10:00 140916.0
2017-07-07 14:20:00 84341.0
2017-07-07 14:30:00 124285.0
2017-07-07 14:40:00 63322.0
2017-07-07 14:50:00 132459.0
2017-07-07 15:00:00 78476.0
2017-07-07 15:10:00 114200.0
2017-07-07 15:20:00 114722.0
2017-07-07 15:30:00 100492.0
2017-07-07 15:40:00 48921.0
2017-07-07 15:50:00 78862.0
2017-07-07 16:00:00 103968.0
2017-07-07 16:10:00 79020.0
2017-07-07 16:20:00 72422.0
2017-07-07 16:30:00 47418.0
2017-07-07 16:40:00 76287.0
2017-07-07 16:50:00 74540.0
2017-07-07 17:00:00 64404.0
2017-07-07 17:10:00 70682.0
2017-07-07 17:20:00 57800.0
2017-07-07 17:30:00 55229.0
2017-07-07 17:40:00 50906.0
2017-07-07 17:50:00 43392.0
2017-07-07 18:00:00 44250.0
2017-07-07 18:10:00 40224.0
2017-07-07 18:20:00 37405.0
2017-07-07 18:30:00 32925.0
2017-07-07 18:40:00 28483.0
2017-07-07 18:50:00 21156.0
2017-07-07 19:00:00 18518.0
2017-07-07 19:10:00 17563.0
2017-07-07 

2017-07-09 22:50:00 0.0
2017-07-09 23:00:00 0.0
2017-07-09 23:10:00 0.0
2017-07-09 23:20:00 0.0
2017-07-09 23:30:00 0.0
2017-07-09 23:40:00 0.0
2017-07-09 23:50:00 0.0
2017-07-10 00:00:00 0.0
2017-07-10 00:10:00 0.0
2017-07-10 00:20:00 0.0
2017-07-10 00:30:00 0.0
2017-07-10 00:40:00 0.0
2017-07-10 00:50:00 0.0
2017-07-10 01:00:00 0.0
2017-07-10 01:10:00 0.0
2017-07-10 01:20:00 0.0
2017-07-10 01:30:00 0.0
2017-07-10 01:40:00 0.0
2017-07-10 01:50:00 0.0
2017-07-10 02:00:00 0.0
2017-07-10 02:10:00 0.0
2017-07-10 02:20:00 0.0
2017-07-10 02:30:00 0.0
2017-07-10 02:40:00 0.0
2017-07-10 02:50:00 0.0
2017-07-10 03:00:00 0.0
2017-07-10 03:10:00 0.0
2017-07-10 03:20:00 0.0
2017-07-10 03:30:00 0.0
2017-07-10 03:40:00 0.0
2017-07-10 03:50:00 0.0
2017-07-10 04:00:00 0.0
2017-07-10 04:10:00 0.0
2017-07-10 04:20:00 0.0
2017-07-10 04:30:00 0.0
2017-07-10 04:40:00 0.0
2017-07-10 04:50:00 0.0
2017-07-10 05:00:00 0.0
2017-07-10 05:10:00 0.0
2017-07-10 05:20:00 0.0
2017-07-10 05:30:00 0.0
2017-07-10 05:40

...

2017-09-19 22:40:00 85500.0
2017-09-19 22:50:00 85500.0
2017-09-19 23:00:00 85500.0
2017-09-19 23:10:00 85500.0
2017-09-19 23:20:00 85500.0
2017-09-19 23:30:00 85500.0
2017-09-19 23:40:00 85500.0
2017-09-19 23:50:00 42750.0
2017-09-20 00:00:00 0.0
2017-09-20 00:10:00 0.0
2017-09-20 00:20:00 0.0
2017-09-20 00:30:00 0.0
2017-09-20 00:40:00 0.0
2017-09-20 00:50:00 0.0
2017-09-20 01:00:00 0.0
2017-09-20 01:10:00 0.0
2017-09-20 01:20:00 0.0
2017-09-20 01:30:00 0.0
2017-09-20 01:40:00 0.0
2017-09-20 01:50:00 0.0
2017-09-20 02:00:00 0.0
2017-09-20 02:10:00 0.0
2017-09-20 02:20:00 0.0
2017-09-20 02:30:00 0.0
2017-09-20 02:40:00 0.0
2017-09-20 02:50:00 0.0
2017-09-20 03:00:00 0.0
2017-09-20 03:10:00 0.0
2017-09-20 03:20:00 0.0
2017-09-20 03:30:00 0.0
2017-09-20 03:40:00 0.0
2017-09-20 03:50:00 0.0
2017-09-20 04:00:00 0.0
2017-09-20 04:10:00 0.0
2017-09-20 04:20:00 0.0
2017-09-20 04:30:00 0.0
2017-09-20 04:40:00 0.0
2017-09-20 04:50:00 0.0
2017-09-20 05:00:00 0.0
2017-09-20 05:10:00 0.0
2017-09-

2017-09-22 06:40:00 0.0
2017-09-22 06:50:00 0.0
2017-09-22 07:00:00 0.0
2017-09-22 07:10:00 0.0
2017-09-22 07:20:00 0.0
2017-09-22 07:30:00 0.0
2017-09-22 07:40:00 0.0
2017-09-22 07:50:00 0.0
2017-09-22 08:00:00 0.0
2017-09-22 08:10:00 0.0
2017-09-22 08:20:00 0.0
2017-09-22 08:30:00 0.0
2017-09-22 08:40:00 0.0
2017-09-22 08:50:00 0.0
2017-09-22 09:00:00 0.0
2017-09-22 09:10:00 0.0
2017-09-22 09:20:00 0.0
2017-09-22 09:30:00 0.0
2017-09-22 09:40:00 0.0
2017-09-22 09:50:00 0.0
2017-09-22 10:00:00 0.0
2017-09-22 10:10:00 0.0
2017-09-22 10:20:00 0.0
2017-09-22 10:30:00 0.0
2017-09-22 10:40:00 0.0
2017-09-22 10:50:00 0.0
2017-09-22 11:00:00 0.0
2017-09-22 11:10:00 0.0
2017-09-22 11:20:00 0.0
2017-09-22 11:30:00 0.0
2017-09-22 11:40:00 0.0
2017-09-22 11:50:00 0.0
2017-09-22 12:00:00 0.0
2017-09-22 12:10:00 0.0
2017-09-22 12:20:00 0.0
2017-09-22 12:30:00 0.0
2017-09-22 12:40:00 0.0
2017-09-22 12:50:00 0.0
2017-09-22 13:00:00 0.0
2017-09-22 13:10:00 0.0
2017-09-22 13:20:00 0.0
2017-09-22 13:30

2017-09-24 16:40:00 0.0
2017-09-24 16:50:00 0.0
2017-09-24 17:00:00 0.0
2017-09-24 17:10:00 0.0
2017-09-24 17:20:00 0.0
2017-09-24 17:30:00 0.0
2017-09-24 17:40:00 0.0
2017-09-24 17:50:00 0.0
2017-09-24 18:00:00 0.0
2017-09-24 18:10:00 0.0
2017-09-24 18:20:00 0.0
2017-09-24 18:30:00 0.0
2017-09-24 18:40:00 0.0
2017-09-24 18:50:00 0.0
2017-09-24 19:00:00 0.0
2017-09-24 19:10:00 0.0
2017-09-24 19:20:00 0.0
2017-09-24 19:30:00 0.0
2017-09-24 19:40:00 0.0
2017-09-24 19:50:00 0.0
2017-09-24 20:00:00 0.0
2017-09-24 20:10:00 0.0
2017-09-24 20:20:00 0.0
2017-09-24 20:30:00 0.0
2017-09-24 20:40:00 0.0
2017-09-24 20:50:00 0.0
2017-09-24 21:00:00 0.0
2017-09-24 21:10:00 0.0
2017-09-24 21:20:00 0.0
2017-09-24 21:30:00 0.0
2017-09-24 21:40:00 0.0
2017-09-24 21:50:00 0.0
2017-09-24 22:00:00 0.0
2017-09-24 22:10:00 0.0
2017-09-24 22:20:00 0.0
2017-09-24 22:30:00 0.0
2017-09-24 22:40:00 0.0
2017-09-24 22:50:00 0.0
2017-09-24 23:00:00 0.0
2017-09-24 23:10:00 0.0
2017-09-24 23:20:00 0.0
2017-09-24 23:30

2017-09-26 22:50:00 0.0
2017-09-26 23:00:00 0.0
2017-09-26 23:10:00 0.0
2017-09-26 23:20:00 0.0
2017-09-26 23:30:00 0.0
2017-09-26 23:40:00 0.0
2017-09-26 23:50:00 0.0
2017-09-27 00:00:00 0.0
2017-09-27 00:10:00 0.0
2017-09-27 00:20:00 0.0
2017-09-27 00:30:00 0.0
2017-09-27 00:40:00 0.0
2017-09-27 00:50:00 0.0
2017-09-27 01:00:00 0.0
2017-09-27 01:10:00 0.0
2017-09-27 01:20:00 0.0
2017-09-27 01:30:00 0.0
2017-09-27 01:40:00 0.0
2017-09-27 01:50:00 0.0
2017-09-27 02:00:00 0.0
2017-09-27 02:10:00 0.0
2017-09-27 02:20:00 0.0
2017-09-27 02:30:00 0.0
2017-09-27 02:40:00 0.0
2017-09-27 02:50:00 0.0
2017-09-27 03:00:00 0.0
2017-09-27 03:10:00 0.0
2017-09-27 03:20:00 0.0
2017-09-27 03:30:00 0.0
2017-09-27 03:40:00 0.0
2017-09-27 03:50:00 0.0
2017-09-27 04:00:00 0.0
2017-09-27 04:10:00 0.0
2017-09-27 04:20:00 0.0
2017-09-27 04:30:00 0.0
2017-09-27 04:40:00 0.0
2017-09-27 04:50:00 0.0
2017-09-27 05:00:00 0.0
2017-09-27 05:10:00 0.0
2017-09-27 05:20:00 0.0
2017-09-27 05:30:00 0.0
2017-09-27 05:40

2017-09-29 06:30:00 0.0
2017-09-29 06:40:00 0.0
2017-09-29 06:50:00 0.0
2017-09-29 07:00:00 0.0
2017-09-29 07:10:00 0.0
2017-09-29 07:20:00 0.0
2017-09-29 07:30:00 0.0
2017-09-29 07:40:00 0.0
2017-09-29 07:50:00 1664.0
2017-09-29 08:00:00 4044.0
2017-09-29 08:10:00 5731.0
2017-09-29 08:20:00 3492.0
2017-09-29 08:30:00 3973.0
2017-09-29 08:40:00 4330.0
2017-09-29 08:50:00 3622.0
2017-09-29 09:00:00 3248.0
2017-09-29 09:10:00 10886.0
2017-09-29 09:20:00 13860.0
2017-09-29 09:30:00 19371.0
2017-09-29 09:40:00 34732.0
2017-09-29 09:50:00 38089.0
2017-09-29 10:00:00 45518.0
2017-09-29 10:10:00 41167.0
2017-09-29 10:20:00 45385.0
2017-09-29 10:30:00 48904.0
2017-09-29 10:40:00 42221.0
2017-09-29 10:50:00 39856.0
2017-09-29 11:00:00 60053.0
2017-09-29 11:10:00 103868.0
2017-09-29 11:20:00 100490.0
2017-09-29 11:30:00 85572.0
2017-09-29 11:40:00 87512.0
2017-09-29 11:50:00 84287.0
2017-09-29 12:00:00 91212.0
2017-09-29 12:10:00 95645.0
2017-09-29 12:20:00 88725.0
2017-09-29 12:30:00 88076.0
20

## Splitting the dataset

Split the data-set into train, validation and test sets. The train set is used to calculate the regression co-efficients, the validation set is used to check the accuracy of the regression model and parameters can be tweaked such that the validation accuracy is improved. Finally, the test-set is the truly *unseen* data. [I think, this function should be provided - Ishaan]

In [6]:
def split_dataset(featureArray,labels):
    """ 
    Splits data-set into train,val and test
    
    """
    
    data_train = []
    label_train = []
    data_val = []
    label_val = []
    data_test = []
    label_test = []
    for row,idx,label in zip(featureArray,range(featureArray.shape[0]),labels):
        if row[0].day <= 20:
            data_train.append(row)
            label_train.append(label)
        elif row[0].day <=25:
            data_val.append(row)
            label_val.append(label)
        else:
            data_test.append(row)
            label_test.append(label)
    return np.asarray(data_train),label_train,np.asarray(data_val),label_val,np.asarray(data_test),label_test

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def transform_text_labels(featureArray):
    """
    Takes the text based labels and transforms them into a one-hot encoded
    feature matrix, could be appended to original feature array
    """
    # Use the label encoder to convert text based features to ints
    le = LabelEncoder()
    textList = list(featureArray[:,1])
    le.fit(textList)
    intList = le.transform(textList) 
    distinctLabels = list(set(intList))
    n, bins, patches = plt.hist(intList, bins = 20)
    plt.show()
    # Choose relevant class labels
    max_occ = max(n) # Gives the text label with max occurences in the data-set
    relevant_classes = []
    # Generate one-hot encoded matrix
    for item,idx in zip(n,range(len(n))):
        if item > max_occ/2:
            relevant_classes.append(idx)
    transMatrix = np.zeros((len(intList),len(relevant_classes)),dtype = np.int)
    for idx,elem in zip(range(transMatrix.shape[0]),intList):
        if elem in relevant_classes:
            transMatrix[idx][relevant_classes.index(elem)] = 1
    return transMatrix        

## Visualizing the data

It is always good to visualize the correlations between different columns of the data-set. It is also very instructive to see how the label values behave w.r.t each individual feature. 

*Hint : Check the documentation for the scatter_matrix from the pandas package*

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#Merge energy and solar energy produced into feature array
mergedMatrix = np.c_[featureArray,labels_solar]
#Create a pandas dataframe for viz
scatterMatrix = mergedMatrix[:,2:].astype(float)
df = pd.DataFrame(data = scatterMatrix
                  #columns = ['Temp','humidity',
                  #            'wind speed','visibility',
                  #            'rain','Day Hour',
                  #            'net energy','solar']
                 )

pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal='kde')

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f554548ac50>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7f553f551518>]], dtype=object)

In [ ]:
transMatrix = transform_text_labels(featureArray)
merge = np.c_[featureArray,transMatrix]
merge_hot = np.c_[transMatrix,labels_solar]
print(merge.shape)
scatterMatrix = merge_hot.astype(float)
df = pd.DataFrame(data = scatterMatrix)

pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal='kde')


## Creating a regression model

At this point, we have a clean feature matrix (split into 3 sets) with the corresponding set of labels. Define a helper function to calculate the prediction error for validations and test sets. Define another function that normalizes the feature vector columns. 

Finally use the regression model from scikit-learn to fit training data. Tweak parameters and check their effect by calculating the prediction error for the validation set.

When the best set of parameters are found, check the prediction error on the test-set.

In [ ]:
def prediction_error(labels_predict,labels_true):
    """
    Calculates the prediction error as the average absolute difference 
    between predicted and true labels
    
    """
    error = 0
    for predicted,true in zip(labels_predict,labels_true):
        #print('True : {} Predicted : {}'.format(true,predicted))
        error += abs(predicted-true)
    
    error = error/len(labels_predict)
    return error

def train_data_stats(data_train):
    return np.mean(data_train.astype(float),axis = 0),np.std(data_train.astype(float), axis = 0)

def normalize_data(data,mean,var):
    """
    Scales data to [-1,1], to be done before feeding it for regression
    
    """  
    norm_data = (data - mean)/var
    return norm_data
    

#Split the dataset
data_train,labels_train,data_val,labels_val,data_test,labels_test = split_dataset(featureArray,labels_solar)

# SGD Regression (Don't normalize the one hot encoded labels !)
mean,var = train_data_stats(data_train[:,2:].astype(float))
X_train = normalize_data(data_train[:,2:].astype(float),mean,var)
X_val = normalize_data(data_val[:,2:].astype(float),mean,var)
X_test = normalize_data(data_test[:,2:].astype(float),mean,var)
clf = linear_model.SGDRegressor(max_iter = 10**6/X_train.shape[0],
                                tol = 1e-6,
                                alpha = 0.1,
                                )
clf.fit(X_train,np.divide(labels_train,1000))

print(clf.coef_)
print(clf.score(X_train,np.divide(labels_train,1000)))
print(clf.score(X_val,np.divide(labels_val,1000)))
print(clf.score(X_test,np.divide(labels_test,1000)))



In [ ]:
print(np.mean(data_train[:,2:].astype(float),axis = 0))
print(np.std(data_train[:,2:].astype(float), axis = 0))

In [ ]:
# Ridge Regression
ridge = linear_model.Ridge(alpha=10.0)
ridge.fit(X_train,np.divide(labels_train,1000))
print(ridge.coef_)
print(ridge.score(X_train,np.divide(labels_train,1000)))

print(ridge.score(X_val,np.divide(labels_val,1000)))
print(ridge.score(X_test,np.divide(labels_test,1000)))


In [ ]:
# Lasso Regression
lasso = linear_model.Lasso(alpha=0.5)
lasso.fit(X_train,np.divide(labels_train,1000))
print(lasso.coef_)
print(lasso.score(X_train,np.divide(labels_train,1000)))

print(lasso.score(X_val,np.divide(labels_val,1000)))
print(lasso.score(X_test,np.divide(labels_test,1000)))

In [ ]:
#Polynomial Features
poly = PolynomialFeatures(degree = 2)
poly_X_train = poly.fit_transform(X_train)
poly_X_val = poly.fit_transform(X_val)
poly_X_test = poly.fit_transform(X_test)

# Lasso Regression
lasso_poly = linear_model.Lasso(alpha=0.1)
lasso_poly.fit(poly_X_train,labels_train)
print(lasso_poly.coef_)
print(lasso_poly.score(poly_X_train,labels_train))

#print(prediction_error(lasso_poly.predict(poly_X_val),labels_val))
#print(prediction_error(lasso_poly.predict(poly_X_test),labels_test))

print(lasso_poly.score(poly_X_val,labels_val))
print(lasso_poly.score(poly_X_test,labels_test))


In [ ]:
#Plotting test and generalization errors
#Split the dataset
data_train,label_train,data_val,label_val,data_test,label_test = split_dataset(featureArray,labels_solar)
num_train_examples = data_train.shape[0]
num_samples = 200
train_error = []
val_error = []
test_error = []
samples = []
mean,var = train_data_stats(data_train[:,2:].astype(float))
X_train = normalize_data(data_train[:,2:].astype(float),mean,var)
X_val = normalize_data(data_val[:,2:].astype(float),mean,var)
X_test = normalize_data(data_test[:,2:].astype(float),mean,var)
while num_samples < num_train_examples:
    samples.append(num_samples)
    # Lasso Regression
    lasso = linear_model.Lasso(alpha=10.0)
    lasso.fit(X_train[0:num_samples,:],labels_train[0:num_samples])
    train_error.append(lasso.score(X_train[0:num_samples,:],labels_train[0:num_samples]))
    val_error.append(lasso.score(X_val,labels_val))
    test_error.append(lasso.score(X_test,labels_test))
    num_samples += 200

#Plot the curves
plt.plot(samples,train_error,'r',label='Training score')
plt.plot(samples,val_error,'g',label='Validation score')
plt.plot(samples,test_error,'b',label='Testing score')
plt.ylabel('Score')
plt.xlabel('Number of training samples')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) 
plt.show()

In [ ]:
#Plotting test and generalization errors
#Split the dataset
data_train,label_train,data_val,label_val,data_test,label_test = split_dataset(featureArray,labels_solar)
num_train_examples = data_train.shape[0]
num_samples = 200
train_error = []
val_error = []
test_error = []
samples = []
X_train = normalize_data(data_train[:,2:].astype(float))
X_val = normalize_data(data_val[:,2:].astype(float))
X_test = normalize_data(data_test[:,2:].astype(float))
poly = PolynomialFeatures(degree = 2)
poly_X_train = poly.fit_transform(X_train)
poly_X_val = poly.fit_transform(X_val)
poly_X_test = poly.fit_transform(X_test)
while num_samples < num_train_examples:
    samples.append(num_samples)
    # Lasso Regression
    lasso = linear_model.Lasso(alpha=10.0)
    lasso.fit(poly_X_train[0:num_samples,:],labels_train[0:num_samples])
    train_error.append(lasso.score(poly_X_train[0:num_samples,:],labels_train[0:num_samples]))
    val_error.append(lasso.score(poly_X_val,labels_val))
    test_error.append(lasso.score(poly_X_test,labels_test))
    num_samples += 200

#Plot the curves
plt.plot(samples,train_error,'r',label='Training score')
plt.plot(samples,val_error,'g',label='Validation score')
plt.plot(samples,test_error,'b',label='Testing score')
plt.ylabel('Score')
plt.xlabel('Number of training samples')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) 
plt.show()

In [ ]:
train_data = data_train[:,2:].astype(float)
train_data -= np.mean(train_data, axis = 0) 
cov = np.dot(train_data.T, train_data) / train_data.shape[0]
print(cov)

In [ ]:
#Pickle all the data
import pickle

with open('data_train.pkl', 'wb') as f:
    pickle.dump(list(X_train), f)

with open('labels_train.pkl', 'wb') as f:
    pickle.dump(list(labels_train), f)
    
with open('data_val.pkl', 'wb') as f:
    pickle.dump(list(X_val), f)

with open('labels_val.pkl', 'wb') as f:
    pickle.dump(list(labels_val), f)

with open('data_test.pkl', 'wb') as f:
    pickle.dump(list(X_test), f)

with open('labels_test.pkl', 'wb') as f:
    pickle.dump(list(labels_test), f)